# Test Stats (Evidencia) - Preguntas Abiertas


## Dataset Download

In [8]:
import pandas as pd
import numpy as np
import requests
import statsmodels.api as sm 

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url = 'https://www.apradie.com/datos/dataus.csv'
response = requests.get(url, headers=headers)
with open('dataus.csv', 'wb') as f:
    f.write(response.content)

# Import as data frames
dataus = pd.read_csv('dataus.csv')

# Display the first few rows of each DataFrame to confirm
print("usfirms DataFrame:")
display(dataus.head())
display(dataus.info())

usfirms DataFrame:


,firm,year,revenue,ebit,netincome,totalassets,totalliabilities,shortdebt,longdebt,retainedearnings,stockprice,marketvalue,epsp,industry
0,AAP,2001,2517639.0,98212.0,25005.0,1950615.0,1662044.0,58463.0,932022.0,-205294.0,14.405094,0.000,NaN,Trade
1,AAP,2002,3287883.0,201985.0,75967.0,1965225.0,1496869.0,11559.0,724832.0,-140275.0,14.158977,1745366.331,0.043525,Trade
2,AAP,2003,3493696.0,288234.0,172234.0,1983071.0,1351827.0,53305.0,422780.0,-15340.0,23.569340,3004390.809,0.057327,Trade
3,AAP,2004,3770297.0,328758.0,190968.0,2201962.0,1479647.0,51884.0,438300.0,172648.0,25.295056,3211784.154,0.059459,Trade
4,AAP,2005,4264971.0,408492.0,231910.0,2542149.0,1622378.0,82930.0,406040.0,407373.0,37.751481,4701384.545,0.049328,Trade


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5975 entries, 0 to 5974
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   firm              5975 non-null   object 
 1   year              5975 non-null   int64  
 2   revenue           5975 non-null   float64
 3   ebit              5975 non-null   float64
 4   netincome         5975 non-null   float64
 5   totalassets       5974 non-null   float64
 6   totalliabilities  5974 non-null   float64
 7   shortdebt         5971 non-null   float64
 8   longdebt          5972 non-null   float64
 9   retainedearnings  5934 non-null   float64
 10  stockprice        5848 non-null   float64
 11  marketvalue       5975 non-null   float64
 12  epsp              5818 non-null   float64
 13  industry          5975 non-null   object 
dtypes: float64(11), int64(1), object(2)
memory usage: 653.6+ KB


None

### Planteamiento del Modelo

Para la immplementación en código del modelo, hice uso de las LLM's: ChatGPT y DeepSeek

### Pasos para correr el modelo

1. ORDENAR EL DATAFRAME POR EMPRESA Y AÑO → para calcular correctamente los retornos por firma.

2. CALCULAR EL RETORNO LOGARÍTMICO ANUAL POR EMPRESA usando np.log(precio_t / precio_{t-1}).

3. ELIMINAR FILAS SIN RETORNO → la primera observación de cada empresa queda sin retorno.

4. ELIMINAR FILAS CON EPSP FALTANTE → aseguramos que la variable explicativa principal esté completa.

5. CREAR DUMMIES DE INDUSTRIA → convertir la variable categórica industry en variables binarias.

6. CREAR INTERACCIONES EPSP × INDUSTRIA → para evaluar si el efecto de EPSP cambia según la industria.

7. COMBINAR VARIABLES NUMÉRICAS, DUMMIES E INTERACCIONES EN X → preparar matriz de regresores.

8. ASEGURAR QUE X Y Y SEAN NUMÉRICOS → convertir todas las columnas a float y manejar posibles errores.

0. ELIMINAR FILAS CON NA EN X O Y → limpiar los datos antes de ajustar el modelo.

10. CONVERTIR X Y Y A FLOAT64 → garantizar compatibilidad con statsmodels.

11. AGREGAR CONSTANTE A X Y AJUSTAR EL MODELO OLS → ejecutar regresión lineal múltiple.

12. IMPRIMIR RESUMEN DEL MODELO → revisar coeficientes, significancia y métricas del ajuste.

In [18]:
# Ordenamos por empresa y año
dataus = dataus.sort_values(by=['firm', 'year'])

# Calculamos el retorno logarítmico por empresa
dataus['log_return'] = dataus.groupby('firm')['stockprice'].transform(
    lambda x: np.log(x / x.shift(1))
)

# Eliminamos la primera observación de cada empresa (sin retorno)
dataus = dataus.dropna(subset=['log_return'])


In [12]:
# Eliminamos filas con epsp faltante
dataus = dataus.dropna(subset=['epsp'])

# Creamos dummies de industria
industry_dummies = pd.get_dummies(dataus['industry'], drop_first=True)

# Interacciones epsp * industria
for col in industry_dummies.columns:
    dataus[f'epsp_{col}'] = dataus['epsp'] * industry_dummies[col]

# Combinamos todo en el X
X = pd.concat([dataus['epsp'], industry_dummies, dataus.filter(like='epsp_')], axis=1)

# Variable dependiente
y = dataus['log_return']


In [15]:
# --- 1. Asegurar que X y y sean numéricos
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# --- 2. Eliminar cualquier fila con NaN resultante
valid_rows = X.notna().all(axis=1) & y.notna()
X = X.loc[valid_rows]
y = y.loc[valid_rows]

# --- 3. Convertir a numpy float64
X = X.astype(float)
y = y.astype(float)

# --- 4. Ajustar el modelo
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:             log_return   R-squared:                       0.161
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     204.7
Date:                Thu, 16 Oct 2025   Prob (F-statistic):          2.86e-200
Time:                        11:57:13   Log-Likelihood:                -3717.1
No. Observations:                5349   AIC:                             7446.
Df Residuals:                    5343   BIC:                             7486.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0679 

### Interpretación 

ESTE ES UN MODLEO DE REGSIÓN LIENAL MÚTIPLE, DONDE LA VARIABLE DEPENDIENTE ES LOG_RETURN, LA CUAL REPRESENTA EL RETORNO LOGARITMICO DE LAS ACCIONES. 

LA VARIABLE EXPLICATIVA EPSP, MIDE LAS GANANCIAS POR ACCIÓN PREDICHAS. LAS VARIABLES SOFTWARE & DATA Y TRADE, SON VARIABLES DUMMIES ABSTRAIDAS DE LA VARIABLE CATRGÓRICA INDUSTRY, E INDICAN LA PERTENENCIA DE CADA EMPRESA A SU RESPECTIVA INDUSTRIA. 

POR OTRO LADO, LAS VARIABLES ESPS_SOFTWARE & TRADA Y EPSP_TRADE, SON LAS VARAIBLES QUE CONTIENEN LAS INTERACCIONES QUE EVALÚAN SI EL EFECTO EPSP SOBRE EL RETORNO VARÍA ACORDE LA INDUSTRIA. 

LA VARIABLE EPSP, MANTIENEN UNA RELACIÓN POSITIVA Y ESTADISTICAMENTE SIGNIFICATIVA CON LA VARIABLE DEPENDIENTE DEL MODELO, YA QUE SU P VALOR ES MENOR A 0.05, Y SUGIERA QUE POR UNA UNDIDAD DE AUMENTO EN ESPS, EL RETRONO LOGARITMICO TIENDA A AUMENTAR EN PROMEDIO 53.9%. 

EN CAMBIO, LA VARIABLE DUMMY SOFTWARE & DATA, NO ES ESTADSTICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P, ES DE 0.811, INDICA QUE COMPARADO CON LA INDSUTRIA BASE, LAS EMPRESAS DEL SECTOR DE SOFTWARE Y DATA TIENEN UN RETORNO LOGRATIMICO MENOR CUANDO EPSP ES CERO. 

LA VARIABLE DUMMY TRADE, DE IGUAL FORMA NO ES ESTADISITICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P ES DE 0.648, Y SUGIERE QUE COMPARADO CON LA INDUSTRIA BASE, EL RETORNO LOGARITMICO DE TRADE ES LIGERAMENTE MENOR EN 0.9%. 

EN CUANTO A LAS INTERACCIONES, LA VARIABLE EPSP × SOFTWARE & DATA NO ES ESTADÍSTICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P ES DE 0.610, LO QUE INDICA QUE EL EFECTO DE EPSP SOBRE EL RETORNO LOGARÍTMICO NO DIFERE SIGNIFICATIVAMENTE ENTRE EL SECTOR SOFTWARE & DATA Y LA INDUSTRIA BASE.

EN CAMBIO, LA VARIABLE EPSP × TRADE ES ESTADÍSTICAMENTE SIGNIFICATIVA (P < 0.001) Y MUESTRA QUE EL EFECTO DE EPSP EN EL SECTOR TRADE ES MÁS DÉBIL QUE EN LA INDUSTRIA BASE, CON UN EFECTO MARGINAL SOBRE EL RETORNO LOGARÍTMICO DE APROXIMADAMENTE 0.1966, AUN POSITIVO PERO CONSIDERABLEMENTE MENOR QUE EN LA INDUSTRIA DE REFERENCIA.


## Extra Points

Para esta implementación de código, se utlizo como LLM ChatGPT

SE UTILIZÓ LA TÉCNICA DE RESIDUOS STUDENTIZADOS EXTERNOS PARA IDENTIFICAR OUTLIERS INFLUYENTES EN EL MODELO, YA QUE ESTOS RESIDUOS MIDEN CUÁNTO SE ALEJA CADA OBSERVACIÓN DEL MODELO AJUSTADO CONSIDERANDO SU INFLUENCIA EN LOS PARÁMETROS, Y AQUELLAS OBSERVACIONES CON VALORES ABSOLUTOS MAYORES A 3 SE CONSIDERAN INUSUALES O INFLUYENTES, PERMITIENDO ELIMINARLAS Y REAJUSTAR EL MODELO PARA EVALUAR SI LOS COEFICIENTES Y LA SIGNIFICANCIA CAMBIAN AL REDUCIR EL EFECTO DE ESTOS PUNTOS EXTREMOS.

In [17]:
# --- 1. Ajuste del modelo original (ya lo tienes)
X_const = sm.add_constant(X)
model = sm.OLS(y, X_const).fit()

# --- 2. Calcular studentized residuals
influence = model.get_influence()
studentized_resid = influence.resid_studentized_external

# --- 3. Identificar outliers: |residual| > 3
outliers = np.where(np.abs(studentized_resid) > 3)[0]
print(f"Número de outliers detectados: {len(outliers)}")

# --- 4. Eliminar outliers usando iloc (por posición)
X_clean = X_const.iloc[~np.isin(np.arange(len(X_const)), outliers)]
y_clean = y.iloc[~np.isin(np.arange(len(y)), outliers)]

# --- 5. Re-ajustar modelo sin outliers
model_clean = sm.OLS(y_clean, X_clean).fit()
print(model_clean.summary())

Número de outliers detectados: 82
                            OLS Regression Results                            
Dep. Variable:             log_return   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     180.4
Date:                Thu, 16 Oct 2025   Prob (F-statistic):          8.96e-178
Time:                        12:24:26   Log-Likelihood:                -2870.2
No. Observations:                5267   AIC:                             5752.
Df Residuals:                    5261   BIC:                             5792.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------

### Interpreteación

ESTE ES UN MODELO DE REGRESIÓN LINEAL MÚLTIPLE, DONDE LA VARIABLE DEPENDIENTE ES LOG_RETURN, LA CUAL REPRESENTA EL RETORNO LOGARÍTMICO DE LAS ACCIONES.

LA VARIABLE EXPLICATIVA EPSP, MIDE LAS GANANCIAS POR ACCIÓN PREDICHAS. LAS VARIABLES SOFTWARE & DATA Y TRADE, SON VARIABLES DUMMIES ABSTRAIDAS DE LA VARIABLE CATGRÓRICA INDUSTRY, E INDICAN LA PERTENENCIA DE CADA EMPRESA A SU RESPECTIVA INDUSTRIA.

POR OTRO LADO, LAS VARIABLES EPSP_SOFTWARE & DATA Y EPSP_TRADE, SON LAS VARIABLES QUE CONTIENEN LAS INTERACCIONES QUE EVALÚAN SI EL EFECTO EPSP SOBRE EL RETORNO VARÍA ACORDE A LA INDUSTRIA.

LA VARIABLE EPSP, MANTIENE UNA RELACIÓN POSITIVA Y ESTADÍSTICAMENTE SIGNIFICATIVA CON LA VARIABLE DEPENDIENTE DEL MODELO, YA QUE SU P-VALOR ES MENOR A 0.05, Y SUGIERE QUE POR UNA UNIDAD DE AUMENTO EN EPSP, EL RETORNO LOGARÍTMICO TIENDE A AUMENTAR EN PROMEDIO 52.9%.

EN CAMBIO, LA VARIABLE DUMMY SOFTWARE & DATA, NO ES ESTADÍSTICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P ES DE 0.675, INDICA QUE COMPARADO CON LA INDUSTRIA BASE, LAS EMPRESAS DEL SECTOR SOFTWARE & DATA TIENEN UN RETORNO LOGARÍTMICO LIGERAMENTE MAYOR CUANDO EPSP ES CERO, PERO LA DIFERENCIA NO ES SIGNIFICATIVA.

LA VARIABLE DUMMY TRADE, DE IGUAL FORMA NO ES ESTADÍSTICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P ES DE 0.667, Y SUGIERE QUE COMPARADO CON LA INDUSTRIA BASE, EL RETORNO LOGARÍTMICO DE TRADE ES LIGERAMENTE MENOR, PERO SIN SIGNIFICANCIA ESTADÍSTICA.

EN CUANTO A LAS INTERACCIONES, LA VARIABLE EPSP × SOFTWARE & DATA NO ES ESTADÍSTICAMENTE SIGNIFICATIVA, YA QUE SU VALOR P ES DE 0.343, LO QUE INDICA QUE EL EFECTO DE EPSP SOBRE EL RETORNO LOGARÍTMICO NO DIFERE SIGNIFICATIVAMENTE ENTRE EL SECTOR SOFTWARE & DATA Y LA INDUSTRIA BASE.

EN CAMBIO, LA VARIABLE EPSP × TRADE NO ES ESTADÍSTICAMENTE SIGNIFICATIVA (P = 0.130), LO QUE SUGIERE QUE EL EFECTO DE EPSP EN EL SECTOR TRADE NO DIFERE SIGNIFICATIVAMENTE DEL EFECTO EN LA INDUSTRIA BASE, CON UN EFECTO MARGINAL SOBRE EL RETORNO LOGARÍTMICO DE APROXIMADAMENTE 0.4486, AUN POSITIVO.